<a href="https://colab.research.google.com/github/Harry-Sui/Predict-if-a-person-is-prone-to-a-heart-attack/blob/main/Anime%20Recommender%20System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing Required Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Load data
rating_data = pd.read_csv("/content/animelist.csv",nrows=50000000)
anima_data = pd.read_csv("/content/anime.csv")

# To save my Pc time I made a new data where i just fetch anime_id(MAL_ID) and Name so that i can use merge() function on it.
anima_data = anima_data.rename(columns={"MAL_ID": "anime_id"})
anima_contact_data = anima_data[["anime_id", "Name"]]

In [ ]:
# Get data information
anima_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17562 entries, 0 to 17561
Data columns (total 35 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   anime_id       17562 non-null  int64 
 1   Name           17562 non-null  object
 2   Score          17562 non-null  object
 3   Genres         17562 non-null  object
 4   English name   17562 non-null  object
 5   Japanese name  17562 non-null  object
 6   Type           17562 non-null  object
 7   Episodes       17562 non-null  object
 8   Aired          17562 non-null  object
 9   Premiered      17562 non-null  object
 10  Producers      17562 non-null  object
 11  Licensors      17562 non-null  object
 12  Studios        17562 non-null  object
 13  Source         17562 non-null  object
 14  Duration       17562 non-null  object
 15  Rating         17562 non-null  object
 16  Ranked         17562 non-null  object
 17  Popularity     17562 non-null  int64 
 18  Members        17562 non-n

In [ ]:
anima_contact_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17562 entries, 0 to 17561
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   anime_id  17562 non-null  int64 
 1   Name      17562 non-null  object
dtypes: int64(1), object(1)
memory usage: 274.5+ KB


In [ ]:
# Merge the rating data and anima_cantct_data
rating_data = rating_data.merge(anima_contact_data, left_on = 'anime_id', right_on = 'anime_id', how = 'left')
rating_data = rating_data[["user_id", "Name", "anime_id","rating", "watching_status", "watched_episodes"]]
rating_data.head()

,user_id,Name,anime_id,rating,watching_status,watched_episodes
0,0,Basilisk: Kouga Ninpou Chou,67.0,9.0,1.0,1.0
1,0,Fairy Tail,6702.0,7.0,1.0,4.0
2,0,Gokusen,242.0,10.0,1.0,4.0
3,0,Kuroshitsuji,4898.0,0.0,1.0,1.0
4,0,One Piece,21.0,10.0,1.0,0.0


In [ ]:
# Get the shape of rating data
rating_data.shape

(2376278, 6)

In [ ]:
# Get half of the amount of data and increase running speed.
count = rating_data['user_id'].value_counts()
count1 = rating_data['anime_id'].value_counts()
rating_data = rating_data[rating_data['user_id'].isin(count[count >= 500].index)].copy()
rating_data = rating_data[rating_data['anime_id'].isin(count1[count1 >= 200].index)].copy()

In [ ]:
# Get the shape of rating data
rating_data.shape

(1023338, 6)

In [18]:
# grouping rating_data by Name and rating into a new column movie_ratingCount
combine_movie_rating = rating_data.dropna(axis = 0, subset = ['Name'])
movie_ratingCount = (combine_movie_rating.
     groupby(by = ['Name'])['rating'].
     count().
     reset_index()
     [['Name', 'rating']]
    )
movie_ratingCount.head()

,Name,rating
0,"""Bungaku Shoujo"" Memoire",161
1,"""Bungaku Shoujo"" Movie",301
2,.hack//Roots,181
3,.hack//Sign,378
4,.hack//Tasogare no Udewa Densetsu,159


In [19]:
# Encoding categorical data
user_ids = rating_data["user_id"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
user_encoded2user = {i: x for i, x in enumerate(user_ids)}
rating_data["user"] = rating_data["user_id"].map(user2user_encoded)
n_users = len(user2user_encoded)

anime_ids = rating_data["anime_id"].unique().tolist()
anime2anime_encoded = {x: i for i, x in enumerate(anime_ids)}
anime_encoded2anime = {i: x for i, x in enumerate(anime_ids)}
rating_data["anime"] = rating_data["anime_id"].map(anime2anime_encoded)
n_animes = len(anime2anime_encoded)

print("Num of users: {}, Num of animes: {}".format(n_users, n_animes))
print("Min total rating: {}, Max total rating: {}".format(min(rating_data['rating']), max(rating_data['rating'])))

Num of users: 1524, Num of animes: 2661
Min total rating: 0.0, Max total rating: 10.0


In [20]:
# Encoding categorical data
g = rating_data.groupby('user_id')['rating'].count()
top_users = g.dropna().sort_values(ascending=False)[:20]
top_r = rating_data.join(top_users, rsuffix='_r', how='inner', on='user_id')

g = rating_data.groupby('anime_id')['rating'].count()
top_animes = g.dropna().sort_values(ascending=False)[:20]
top_r = top_r.join(top_animes, rsuffix='_r', how='inner', on='anime_id')

pivot = pd.crosstab(top_r.user_id, top_r.anime_id, top_r.rating, aggfunc=np.sum)

In [21]:
pivot.fillna(0, inplace=True)
pivot

anime_id,20.0,30.0,226.0,849.0,1535.0,1575.0,2001.0,2167.0,2904.0,3588.0,...,23273.0,28121.0,28171.0,28223.0,30276.0,30831.0,31043.0,31240.0,31964.0,32281.0
user_id,,,,,,,,,,,,,,,,,,,,,
240,5.0,8.0,0.0,0.0,8.0,8.0,8.0,4.0,8.0,0.0,...,9.0,6.0,7.0,0.0,6.0,4.0,6.0,8.0,5.0,6.0
1397,0.0,10.0,7.0,8.0,9.0,9.0,8.0,7.0,9.0,9.0,...,8.0,7.0,8.0,7.0,7.0,8.0,8.0,9.0,8.0,10.0
1469,7.0,2.0,8.0,3.0,7.0,9.0,3.0,5.0,9.0,2.0,...,5.0,9.0,9.0,8.0,9.0,10.0,10.0,8.0,6.0,8.0
1946,8.0,10.0,9.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,...,9.0,8.0,8.0,8.0,9.0,8.0,9.0,9.0,8.0,8.0
2465,0.0,0.0,9.0,0.0,0.0,10.0,0.0,0.0,10.0,0.0,...,0.0,8.0,9.0,9.0,10.0,9.0,8.0,10.0,8.0,10.0
3021,0.0,9.0,7.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,...,0.0,0.0,9.0,0.0,0.0,9.0,0.0,8.0,0.0,8.0
3160,10.0,0.0,0.0,0.0,8.0,0.0,0.0,10.0,0.0,0.0,...,9.0,8.0,0.0,10.0,0.0,8.0,0.0,0.0,10.0,10.0
3578,9.0,9.0,10.0,10.0,10.0,10.0,9.0,10.0,10.0,10.0,...,8.0,7.0,9.0,0.0,8.0,8.0,9.0,8.0,7.0,0.0
4132,9.0,0.0,7.0,0.0,10.0,7.0,0.0,6.0,0.0,5.0,...,8.0,7.0,9.0,7.0,7.0,5.0,9.0,9.0,8.0,7.0


In [22]:
# Creat a piviot table based on Name and User_id column and save it into a variable name piviot_table
piviot_table = rating_data.pivot_table(index="Name",columns="user_id", values="rating").fillna(0)
piviot_table

user_id,6,12,17,19,21,42,44,47,53,60,...,7912,7915,7916,7924,7928,7929,7931,7932,7934,7951
Name,,,,,,,,,,,,,,,,,,,,,
"""Bungaku Shoujo"" Memoire",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0
"""Bungaku Shoujo"" Movie",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,7.0,7.0,0.0,0.0,10.0,0.0,0.0
.hack//Roots,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
.hack//Sign,0.0,10.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
.hack//Tasogare no Udewa Densetsu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
xxxHOLiC,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,9.0,0.0,...,8.0,0.0,9.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
xxxHOLiC Kei,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,...,10.0,0.0,9.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
xxxHOLiC Movie: Manatsu no Yoru no Yume,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
# Importing Required Libraries
from scipy.sparse import csr_matrix
piviot_table_matrix = csr_matrix(piviot_table.values)

In [24]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(metric="cosine", algorithm="brute")
model.fit(piviot_table_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [25]:
# Define predict model
def predict():
    random_anime = np.random.choice(piviot_table.shape[0]) # This will choose a random anime name and our model will predict on it.

    query = piviot_table.iloc[random_anime, :].values.reshape(1, -1)
    distance, suggestions = model.kneighbors(query, n_neighbors=6)
    
    for i in range(0, len(distance.flatten())):
        if i == 0:
            print('Recommendations for {0}:\n'.format(piviot_table.index[random_anime]))
        else:
            print('{0}: {1}, with distance of {2}:'.format(i, piviot_table.index[suggestions.flatten()[i]], distance.flatten()[i]))

In [26]:
# Result of Recommender system
predict()

Recommendations for Ashita no Joe:

1: Hajime no Ippo, with distance of 0.7182147393731257:
2: Ginga Eiyuu Densetsu, with distance of 0.7196123202481403:
3: Hajime no Ippo: Rising, with distance of 0.7284422070765874:
4: Hajime no Ippo: New Challenger, with distance of 0.7323584331452:
5: Versailles no Bara, with distance of 0.7329865753045353:
